In [36]:
## Mount from professor's drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**PROBLEM 1**

In [44]:
## bestK() FUNCTION
def bestK(X, Y, maxK, maxRow):
  
  # Get predictors of knn from X and Y and set allK = True, leave1out = True
  knnout, nn = kNN(X, Y, X, maxK, allK=True, leave1out=True)

  # Create a list for all best K values and MAPE
  bestk_values = []
  MAPE_values = []

  # Loop through the rows and columns of data based on maxRows and knnout predictors to get MAPE and besk K values
  for i in range(0, maxRow):
    for j in range(0, len(knnout)):
      # Get Y predictors and outcomes 
      Y_pred = knnout[i][j]
      Y_outcome = Y.iloc[i]

      # Get MAPE values
      MAPE = findOverallLoss(Y_pred, Y_outcome)
      MAPE_values.append(MAPE)

    # Get best K value by the index of the lowest MAPE value
    bestk = MAPE_values.index(min(MAPE_values))
    bestk_values.append(bestk)
  
  print(bestk_values)

  # Get the most common K values:
  from scipy import stats as s
  most_common_k = int(s.mode(bestk_values)[0])
  print(most_common_k)

## findOverallLoss() FUNCTION
def findOverallLoss(ypreds, Y):
  err = abs(ypreds - Y)
  MAPE = err.mean()
  return MAPE

## kNN() FUNCTION
def kNN(X,Y,newx,k,regress=True,allK=False,leave1out=False,scaleX=True,scaler=StandardScaler()):

  import warnings
  warnings.filterwarnings('ignore')

  import numpy as np

  from sklearn.neighbors import KNeighborsClassifier 
  from sklearn.neighbors import KNeighborsRegressor

  from sklearn.preprocessing import StandardScaler
  from sklearn.preprocessing import MinMaxScaler
  from sklearn.preprocessing import RobustScaler

  from statistics import mean 
  from statistics import mode
  from collections import Counter

  def kNNtype(neighbs,regress):
    if regress:
      knn = KNeighborsRegressor(n_neighbors=neighbs)
    else:
      knn = KNeighborsClassifier(n_neighbors=neighbs)
    return knn

  if scaler != StandardScaler():
    scaler=scaler


# Update: for row subsets/test sets in hw; fitting the scaling function should be done separate,
# so the same can be applied to train and test data (or X and newx) 
  if scaleX == True:
    # scale should be fit to X/train
    scaler.fit(X)
    columns_X = [list(X.columns)]
    for feature in columns_X:
      X[feature] = scaler.transform(X[feature])
      newx[feature] = scaler.transform(newx[feature])
  

  knn_all = pd.DataFrame()
  if allK == True:
    if leave1out == True:
      nn_all = []
      for j in list(newx.index.values.tolist()): #(4/13)
      #for j in list(Y.index.values.tolist()): 
        knn_row = []
        knn = kNNtype(k+1,regress)
        knn.fit(X, Y)
        test = pd.DataFrame(newx.loc[j,:])
        nn = knn.kneighbors(test.T)[1][0]
        for i in range(2,k+1):
          nn1 = nn[1:i] # leave one out
          test = list(Y.iloc[nn1])
          if regress:
            test = mean(test)
          else:
            c = Counter(test)
            l = list(c.values())
            ind = l.index(max(c.values()))
            test = list(c.keys())[ind]
            # count number of times the max class occurs and if there is a tie
            # choose the second class with the max if index is even
            if (l.count(max(l))) > 1 and (j % 2 !=0):
              l[ind] = 0
              ind = l.index(max(c.values()))
              test = list(c.keys())[ind]

          knn_row.append(test)
        knn_row = pd.DataFrame(knn_row)
        knn_all = [knn_all, knn_row]
        knn_all = pd.concat(knn_all,axis=1, ignore_index=True)
        nn_all.append(list(nn1))
      nn_all = np.array(nn_all)
    else:
        for i in range(1,k+1):
          knn = kNNtype(i,regress)
          knn.fit(X, Y)
          test = knn.predict(newx)
          knn_row = pd.DataFrame(test).T
          knn_all = [knn_all, knn_row]
          knn_all = pd.concat(knn_all,axis=0, ignore_index=True)
        nn_all = knn.kneighbors(newx)[1]
  else:
    if leave1out == True:
      knn_row = []
      #for j in list(Y.index.values.tolist()):
      for j in list(newx.index.values.tolist()): # (4/13)
        knn = kNNtype(k,regress)
        knn.fit(X, Y)
        test = pd.DataFrame(newx.loc[j,:])
        nn = knn.kneighbors(test.T)[1][0]
        nn1 = nn[1:len(nn)]

        test = list(Y.iloc[nn1])
        if regress:
            test = mean(test)
        else:
          c = Counter(test)
          l = list(c.values())
          ind = l.index(max(c.values()))
          test = list(c.keys())[ind]
          # count number of times the max class occurs and if there is a tie
          # choose the second class with the max if index is even
          if (l.count(max(l))) > 1 and (j % 2 !=0):
            l[ind] = 0
            ind = l.index(max(c.values()))
            test = list(c.keys())[ind]

        knn_row.append(test)
      knn_all = pd.DataFrame(knn_row).T
      nn_all = nn1
    else:
        knn = kNNtype(k,regress)
        knn.fit(X, Y)
        test = knn.predict(newx)
        knn_all = pd.DataFrame(test)
        nn_all = knn.kneighbors(newx)[1]

  return knn_all, nn_all

## MAIN CODES ##
my_path = '/content/drive/My Drive/ecs171_yancey/Lecture_Notes/Chapter_1/day1.csv'
import pandas as pd
day1 = pd.read_csv(my_path)

day1 = day1.loc[:,["workingday","temp","atemp","hum","windspeed","tot"]]
X = day1.iloc[:,0:5]
Y = day1.tot

bestK(X, Y, 30, 5)

[17, 32, 58, 58, 116]
58


**PROBLEM 2**

In [39]:
## p_hack() FUNCTION
def p_hack(X, Y, n_subsets, maxK):

  # Since there are n subsets to analyze, we get the length of each subset by length of X divided by n_subsets
  import math 
  subset_length = math.floor(len(X) / n_subsets)

  # Create list for besk K values and MAPE values
  bestk_values = []
  MAPE_values = []
  bestk_of_subset = []

  # Loop through X and Y to get each subset from them
  for i in range(0, n_subsets):
    # Get start and end indices of the subset
    start = i * subset_length
    end = (i+1) * subset_length

    # Get the subset of X and Y
    subset_X = X[start:end]
    subset_Y = Y[start:end]

    # Get predictors of knn from X and Y and set allK = True, leave1out = True
    knnout, nn = kNN(subset_X, subset_Y, subset_X, maxK, allK=True,leave1out=True)

    # Loop through each subset of Y to find the best K values
    for row in range(0, len(subset_Y)): 
      for col in range(len(knnout)):
        Y_pred = knnout[row][col] # Get predicted values of Y
        Y_outcome = Y.iloc[row] # Get outcomes of Y

        # Get MAPE values
        MAPE = findOverallLoss(Y_pred, Y_outcome)
        MAPE_values.append(MAPE)

      # Find best K values with the lowest MAPE value (lowest loss)
      bestk = MAPE_values.index(min(MAPE_values))
      bestk_values.append(bestk)

    # Get the most common K using mode and add to bestK for a subset of X
    from scipy import stats as s
    most_common_k = int(s.mode(bestk_values)[0])
    bestk_of_subset.append(most_common_k)

  print(bestk_of_subset)
  
  # Calculate variance from best K values of each subset
  import statistics
  variance = statistics.variance(bestk_of_subset)
  print(variance)

## findOverallLoss() FUNCTION
def findOverallLoss(ypreds, Y):
  err = abs(ypreds - Y)
  MAPE = err.mean()
  return MAPE

## MAIN CODES ##
my_path = '/content/drive/My Drive/ecs171_yancey/Lecture_Notes/Chapter_1/day1.csv'
day1 = pd.read_csv(my_path)
day1 = day1.loc[:,["workingday","temp","atemp","hum","windspeed","tot"]]
X = day1.iloc[:,0:5]
Y = day1.tot

p_hack(X, Y, 30, 5)

[22, 93, 93, 93, 93, 93, 93, 93, 93, 93, 93, 93, 93, 93, 93, 93, 93, 93, 93, 93, 93, 93, 93, 93, 93, 93, 93, 93, 93, 93]
168.03333333333333


**PROBLEM 3**

In [0]:
## Should be run in RStudio Code

library(regtools)
data(day)
# head(day)
d <- day[c('temp','atemp','hum','windspeed','casual','registered','cnt')]
lmout <- lm(cnt ~ .,data=d)
xd = preprocessx(d[,-5],10)
knnout <- knnest(d$cnt,xd,10)
parvsnonparplot(lmout,knnout)

**PROBLEM 4**

In [45]:
## atRisk(rows,k,path)
def atRisk(rows,k,path):
  import pandas as pd
  vert = pd.read_csv(path,sep=' ',header=None)
    
  # get X and Y data
  X = vert.iloc[:,0:6]
  from sklearn.preprocessing import LabelEncoder
  le = LabelEncoder()
  vert.iloc[:,6] = le.fit_transform(vert.iloc[:,6])
  Y = vert.iloc[:,6]

  # Get knn via Classifier and fit the knn with X and Y
  from sklearn.neighbors import KNeighborsClassifier
  knn = KNeighborsClassifier(n_neighbors=k)
  knn.fit(X, Y)
    
  # Get test data and predict probabilities
  test = pd.DataFrame(X.iloc[0:11,:])
  array = knn.predict_proba(test)

  # Loop through 2D array to check categories and replace values: 
  for i in range(0,len(array)):
    for j in range(0,len(array[0])):
      if array[i][j] < 0.3:
        array[i][j] = 0
      elif array[i][j] >= 0.3 and array[i][j] < 0.5: 
        array[i][j] = 2
      elif array[i][j] >= 0.5:
        array[i][j] = 1
    
  # Display array
  print(array)


## MAIN CODES ## 
my_path = "/content/drive/My Drive/ecs171_yancey/Lecture_Notes/Chapter_2/column_3C.dat"
atRisk(30, 5, my_path)

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [2. 1. 0.]
 [2. 1. 0.]
 [1. 2. 0.]
 [1. 2. 0.]
 [1. 0. 0.]
 [1. 2. 0.]]
